# EM shower reconstruction with the SciFi at SND@LHC

1. __make sure the preprocessing has already been done__

2. __make sure `results` folder exists__


based on https://github.com/pauldebryas/SND_trackers_My_data/tree/SNDatLHC_neutrino

In [1]:
#!/usr/bin/env python3
# Import Class from utils.py & net.py file
from utils import DataPreprocess, Parameters
from net import SNDNet, BNN, MyDataset, digitize_signal

import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd
from matplotlib import pylab as plt
import time
from tqdm import tqdm
from IPython import display

import os
import gc  # Gabage collector interface (to debug stuff)
import sys

import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
from pyro.infer.autoguide import AutoDiagonalNormal
from pyro.infer import SVI, Trace_ELBO, Predictive

Welcome to JupyROOT 6.18/00


In [2]:
# Test to see if cuda is available or not + listed the CUDA devices that are available
try:
    assert(torch.cuda.is_available())
except:
    raise Exception("CUDA is not available")
    
n_devices = torch.cuda.device_count()
print("\nWelcome!\n\nCUDA devices available:\n")

for i in range(n_devices):
    print("\t{}\twith CUDA capability {}".format(torch.cuda.get_device_name      (device=i), 
                                                 torch.cuda.get_device_capability(device=i)))
print("\n")

device = torch.device("cuda", 0)


Welcome!

CUDA devices available:

	Quadro P2200	with CUDA capability (6, 1)




## Data processing

In [3]:
# Turn off interactive plotting: for long run it screws up everything
plt.ioff()

# Here we choose the geometry with 9 time the radiation length
params = Parameters("SNDatLHC")  #!!!!!!!!!!!!!!!!!!!!!CHANGE THE DIMENTION !!!!!!!!!!!!!!!!
processed_file_path_1 = os.path.expandvars("$HOME/snd_data/processed_data/CCDIS")
processed_file_path_2 = os.path.expandvars("$HOME/snd_data/processed_data/NuEElastic")

In [4]:
# --- LOAD THE reindex_TT_df & reindex_y_full PD.DATAFRAME ---

chunklist_TT_df_CCDIS = []  # list of the TT_df file of each chunk
chunklist_TT_df_NueEElastic = []
chunklist_y_full_CCDIS = [] # list of the y_full file of each chunk
chunklist_y_full_NueEElastic = []

# It is reading and analysing data by chunk instead of all at the time (memory leak problem)
print("\nReading the tt_cleared.pkl & y_cleared.pkl files by chunk of CCDIS and NueEElastic")


Reading the tt_cleared.pkl & y_cleared.pkl files by chunk of CCDIS and NueEElastic


In [5]:
# CCDIS

step_size = 1000    # size of a chunk
file_size = 400000  # size of the CCDIS BigFile
n_steps = int(file_size / step_size) # number of chunks

# First 2 
outpath_1 = processed_file_path_1 + "/{}".format(0)
chunklist_TT_df_CCDIS.append(pd.read_pickle(os.path.join(outpath_1, "tt_cleared.pkl")))
chunklist_y_full_CCDIS.append(pd.read_pickle(os.path.join(outpath_1, "y_cleared.pkl")))

outpath_1 = processed_file_path_1 + "/{}".format(1)
chunklist_TT_df_CCDIS.append(pd.read_pickle(os.path.join(outpath_1, "tt_cleared.pkl")))
chunklist_y_full_CCDIS.append(pd.read_pickle(os.path.join(outpath_1, "y_cleared.pkl")))

reindex_TT_df_CCDIS = pd.concat([chunklist_TT_df_CCDIS[0],chunklist_TT_df_CCDIS[1]],ignore_index=True)
reindex_y_full_CCDIS = pd.concat([chunklist_y_full_CCDIS[0],chunklist_y_full_CCDIS[1]], ignore_index=True)

for i in tqdm(range(n_steps-2)):  # tqdm: make your loops show a progress bar in terminal
    outpath_1 = processed_file_path_1 + "/{}".format(i+2)
    # add all the tt_cleared.pkl files read_pickle and add to the chunklist_TT_df list
    chunklist_TT_df_CCDIS.append(pd.read_pickle(os.path.join(outpath_1, "tt_cleared.pkl")))
    # add all the y_cleared.pkl files read_pickle and add to the chunklist_y_full list
    chunklist_y_full_CCDIS.append(pd.read_pickle(os.path.join(outpath_1, "y_cleared.pkl")))
    reindex_TT_df_CCDIS = pd.concat([reindex_TT_df_CCDIS,chunklist_TT_df_CCDIS[i+2]], ignore_index=True)
    reindex_y_full_CCDIS = pd.concat([reindex_y_full_CCDIS,chunklist_y_full_CCDIS[i+2]], ignore_index=True)

100%|██████████| 398/398 [00:06<00:00, 57.47it/s]


In [6]:
#NueEElastic

step_size = 1000    # size of a chunk
file_size = 399000  # size of the NueEElastic BigFile
n_steps = int(file_size / step_size) # number of chunks

#First 2 
outpath_2 = processed_file_path_2 + "/{}".format(0)
chunklist_TT_df_NueEElastic.append(pd.read_pickle(os.path.join(outpath_2, "tt_cleared.pkl")))
chunklist_y_full_NueEElastic.append(pd.read_pickle(os.path.join(outpath_2, "y_cleared.pkl")))

outpath_2 = processed_file_path_2 + "/{}".format(1)
chunklist_TT_df_NueEElastic.append(pd.read_pickle(os.path.join(outpath_2, "tt_cleared.pkl")))
chunklist_y_full_NueEElastic.append(pd.read_pickle(os.path.join(outpath_2, "y_cleared.pkl")))

reindex_TT_df_NueEElastic = pd.concat([chunklist_TT_df_NueEElastic[0],chunklist_TT_df_NueEElastic[1]],ignore_index=True)
reindex_y_full_NueEElastic = pd.concat([chunklist_y_full_NueEElastic[0],chunklist_y_full_NueEElastic[1]], ignore_index=True)

for i in tqdm(range(n_steps-2)):  # tqdm: make your loops show a progress bar in terminal
    outpath_2 = processed_file_path_2 + "/{}".format(i+2)
    chunklist_TT_df_NueEElastic.append(pd.read_pickle(os.path.join(outpath_2, "tt_cleared.pkl"))) # add all the tt_cleared.pkl files read_pickle and add to the chunklist_TT_df list
    chunklist_y_full_NueEElastic.append(pd.read_pickle(os.path.join(outpath_2, "y_cleared.pkl"))) # add all the y_cleared.pkl files read_pickle and add to the chunklist_y_full list
    reindex_TT_df_NueEElastic = pd.concat([reindex_TT_df_NueEElastic,chunklist_TT_df_NueEElastic[i+2]], ignore_index=True)
    reindex_y_full_NueEElastic = pd.concat([reindex_y_full_NueEElastic,chunklist_y_full_NueEElastic[i+2]], ignore_index=True)

100%|██████████| 397/397 [00:04<00:00, 98.20it/s] 


In [7]:
'''
print("Before Reduction  :")
print("TT_df inelastic: " + str(len(reindex_TT_df_CCDIS)))
print("y_full inelastic: " + str(len(reindex_y_full_CCDIS)))
print("TT_df elastic: " + str(len(reindex_TT_df_NueEElastic)))
print("y_full elastic: " + str(len(reindex_y_full_NueEElastic)))
'''

'\nprint("Before Reduction  :")\nprint("TT_df inelastic: " + str(len(reindex_TT_df_CCDIS)))\nprint("y_full inelastic: " + str(len(reindex_y_full_CCDIS)))\nprint("TT_df elastic: " + str(len(reindex_TT_df_NueEElastic)))\nprint("y_full elastic: " + str(len(reindex_y_full_NueEElastic)))\n'

In [8]:
# Selecting events to ensure equal number of elastic and inelastic events
event_limit = min(len(reindex_TT_df_CCDIS),len(reindex_TT_df_NueEElastic))

remove = int(len(reindex_TT_df_CCDIS)-event_limit)+1
reindex_TT_df_CCDIS = reindex_TT_df_CCDIS[:-remove]
reindex_y_full_CCDIS = reindex_y_full_CCDIS[:-remove]

remove = int(len(reindex_TT_df_NueEElastic)-event_limit)+1
reindex_TT_df_NueEElastic = reindex_TT_df_NueEElastic[:-remove]
reindex_y_full_NueEElastic = reindex_y_full_NueEElastic[:-remove]

# Merging CCDIS and NueEElastic in a single array
reindex_TT_df = pd.concat([reindex_TT_df_CCDIS,reindex_TT_df_NueEElastic], ignore_index=True)
reindex_y_full = pd.concat([reindex_y_full_CCDIS,reindex_y_full_NueEElastic], ignore_index=True)

In [9]:
'''
print("After Reduction  :")
print("TT_df inelastic: " + str(len(reindex_TT_df_CCDIS)))
print("y_full inelastic: " + str(len(reindex_y_full_CCDIS)))
print("TT_df elastic: " + str(len(reindex_TT_df_NueEElastic)))
print("y_full elastic: " + str(len(reindex_y_full_NueEElastic)))

print("Combined TT_df : " + str(len(reindex_TT_df)))
print("Combined y_full: " + str(len(reindex_y_full)))
'''
'''
# -1 for CCDIS
for i in range(event_limit-1):
    reindex_y_full.iloc[i]["Label"] = -1  
'''

'\n# -1 for CCDIS\nfor i in range(event_limit-1):\n    reindex_y_full.iloc[i]["Label"] = -1  \n'

In [10]:
# reset to empty space
chunklist_TT_df_NueEElastic = []
chunklist_y_full_NueEElastic = []
reindex_TT_df_NueEElastic = []
reindex_y_full_NueEElastic = []

chunklist_TT_df_CCDIS = []
chunklist_y_full_CCDIS = []
reindex_TT_df_CCDIS = []
reindex_y_full_CCDIS = []


nb_of_plane = len(params.snd_params[params.configuration]["TT_POSITIONS"])

# True value of NRJ for each true Nue event
y = reindex_y_full[["E"]]
NORM = 1. / 4000
y["E"] *= NORM

# Spliting
print("\nSplitting the data into a training and a testing sample")

indeces = np.arange(len(reindex_TT_df))
train_indeces, test_indeces, _, _ = train_test_split(indeces, indeces, train_size=0.9, random_state=1543)


Splitting the data into a training and a testing sample


/home/fsergeev/anaconda3/envs/ship_tt/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
# get smaller datasets to make training quicker

debug_coef = 0.2

train_indeces = train_indeces[0 : int(train_indeces.shape[0] * debug_coef)]
test_indeces  = train_indeces[0 : int(test_indeces .shape[0] * debug_coef)]

In [12]:
#batch_size = 512
#batch_size = 128
batch_size = 16

train_dataset = MyDataset(reindex_TT_df, y, params, train_indeces, n_filters=nb_of_plane)
train_batch_gen = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

test_dataset = MyDataset(reindex_TT_df, y, params, test_indeces, n_filters=nb_of_plane)
test_batch_gen = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# reset to empty space
reindex_TT_df=[]

# Saving the true Energy for the test sample
TrueE_test=y["E"][test_indeces]
np.save("results/TrueE_test.npy",TrueE_test)

# Saving the Label for the test sample
TrueLabel_test = reindex_y_full["Label"][test_indeces]
np.save("results/TrueLabel_test.npy",TrueLabel_test)

## Training

In [13]:
for X_batch, y_batch in train_batch_gen:
    print(X_batch[0].shape)
    break

torch.Size([5, 201, 201])


In [14]:
torch.set_default_tensor_type("torch.cuda.FloatTensor")

In [23]:
# Creating the network
net = BNN(n_input_filters=nb_of_plane)
guide = AutoDiagonalNormal(net)
predictive = Predictive(net, guide=guide, num_samples=100, return_sites=("obs", "_RETURN"))

# Loose rate, num epoch and weight decay parameters of our network backprop actions
adam = pyro.optim.Adam({"lr": 1e-3})
svi = SVI(net, guide, adam, loss=Trace_ELBO()) # stochastic variational inference class

num_epochs = 3

train_loss = []
val_accuracy_1 = []
val_accuracy_2 = []

# Create a directory where to store the 9X0 files
os.system("mkdir bayesian_results/9X0_file")

256

mkdir: cannot create directory ‘bayesian_results/9X0_file’: File exists


In [24]:
#Training
print("\nNow Trainig the network:")
# Create a .txt file where we will store some info for graphs
f=open("bayesian_results/NN_performance.txt","a")
f.write("Epoch/Time it took (s)/Loss/Validation energy (%)/Validation distance (%)\n")
f.close()


Now Trainig the network:


In [25]:
class Logger(object):
    def __init__(self):
        pass

    def plot_losses(self, epoch, num_epochs, start_time):
        # Print and save in NN_performance.txt the results for this epoch:
        print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss (in-iteration): \t{:.6f}".format(train_loss[-1]))
        print("  validation Energy:\t\t{:.4f} %".format(val_accuracy_1[-1]))
        #print("  validation distance:\t\t{:.4f} %".format(val_accuracy_2[-1]))

        f=open("bayesian_results/NN_performance.txt","a")
        f.write("{};{:.3f};".format(epoch + 1, time.time() - start_time))
        f.write("\t{:.6f};".format(train_loss[-1]))
        f.write("\t\t{:.4f}\n".format(val_accuracy_1[-1]))
        #f.write("\t\t{:.4f}\n".format(val_accuracy_2[-1]))
        f.close()

In [26]:
def summary(samples):
    site_stats = {}
    for k, v in samples.items():
        site_stats[k] = {
            "mean": torch.mean(v, 0),
            "std":  torch.std(v, 0),
            "5%":   v.kthvalue(int(len(v) * 0.05), dim=0)[0],
            "95%":  v.kthvalue(int(len(v) * 0.95), dim=0)[0],
        }
    return site_stats

In [32]:
def run_training(lr, num_epochs, opt):
    try:
        pyro.clear_param_store()

        total = int(len(train_indeces) / batch_size)
        
        for epoch in range(num_epochs):
            # In each epoch, we do a full pass over the training data:
            start_time = time.time()
            
            guide.requires_grad_(True)
            
            epoch_loss = 0
            for X_batch, y_batch in tqdm(train_batch_gen, total=total):
                loss = svi.step(X_batch.to(device), y_batch.to(device))
                epoch_loss += loss

            train_loss.append(epoch_loss / (len(train_indeces) // batch_size + 1))

            guide.requires_grad_(False)

            y_score = []
            
            with torch.no_grad():
                for (X_batch, y_batch) in tqdm(test_batch_gen, total = int(len(test_indeces) / batch_size)):
                    samples = predictive(X_batch.to(device), y_batch.to(device))
                    pred_summary = summary(samples)
                    bayes_preds = pred_summary["_RETURN"]['mean']
                    
                    y_pred = bayes_preds.cpu().detach().numpy()
                    y_score.extend(y_pred)

            y_score = mean_squared_error(y.iloc[test_indeces], np.asarray(y_score), multioutput='raw_values')
            val_accuracy_1.append(y_score[0])
            #val_accuracy_2.append(y_score[1])    
            
            # Visualize
            display.clear_output(wait=True)
            logger.plot_losses(epoch, num_epochs, start_time)

            #Saving network for each 1 epoch
            if (epoch + 1) % 1 == 0:
                with open("bayesian_results/9X0_file/" + str(epoch) + "_9X0_coordconv.pt", 'wb') as f:
                    torch.save(net, f)       
                # optimizer is inside svi
                # lr = lr / 2
                # opt = torch.optim.Adam(net.model.parameters(), lr=lr)

    except KeyboardInterrupt:
        pass

In [33]:
logger = Logger()
run_training(lr=None, num_epochs=num_epochs, opt=None)

Epoch 3 of 3 took 157.573s
  training loss (in-iteration): 	1888.426353
  validation Energy:		7.1178 %


In [37]:
# Saving the prediction at each epoch

# Create a directory where to store the prediction files
os.system("mkdir bayesian_results/PredE_file")

for i in [0,1,2]:#[9, 19, 29, 39, 49]:
    net = torch.load("bayesian_results/9X0_file/" + str(i) + "_9X0_coordconv.pt")
    preds = []
    
    with torch.no_grad():
        for (X_batch, y_batch) in tqdm(test_batch_gen):
            samples = predictive(X_batch.to(device), y_batch.to(device))
            pred_summary = summary(samples)
            bayes_preds = pred_summary["_RETURN"]['mean']

            y_pred = bayes_preds.cpu().detach().numpy()
            preds.append(y_pred)
    
    np.save("bayesian_results/PredE_file/" + str(i) + "_PredE_test.npy", preds)
    print("Save Prediction for epoch "+ str(i))

  0%|          | 0/83 [00:00<?, ?it/s]

Save Prediction for epoch 0


  0%|          | 0/83 [00:00<?, ?it/s]

Save Prediction for epoch 1


100%|██████████| 83/83 [01:36<00:00,  1.16s/it]

Save Prediction for epoch 2



mkdir: cannot create directory ‘bayesian_results/PredE_file’: File exists
